Import Libraries

In [ ]:
import scipy as sp
from scipy import optimize
from scipy.optimize import fsolve
from scipy import interpolate

# Import pandas
import pandas as pd

# Import numpy
import numpy as np
from numpy import * 
from numpy.linalg import multi_dot
import numpy_financial as npf

# Plot settings
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = 16, 8

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# IBT or Pension or Retirement
PLAN = 'IBT'

# % Probability of Sponsor Not Contributing Service Costs
UPS_Contr_Pctg = .05

# Target Years to be "Fully Funded"
Years_to_FF = 20

# Target Fully Funded Ratio
FF_FR = 1.05

In [ ]:
df_CF = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "PBO Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF

In [ ]:
df_CF = df_CF.divide(12)
df_CF

In [ ]:
df=df_CF.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_CF = df
df_CF

In [ ]:
df_CF_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "CF", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_CF_add

In [ ]:
df_CF  = df_CF_add.append(df_CF)
df_CF

In [ ]:
from itertools import count, takewhile
def frange(start, stop, step):
        return takewhile(lambda x: x< stop, count(start, step))
t = list(frange((1/12), 79.41666666666, (1/12)))

In [ ]:
df_t=pd.DataFrame(t)
df_t

In [ ]:
df_CF['Time'] = df_t.values
df_CF

In [ ]:
#IBT = df_CF[PLAN]
#RP = df_CF[PLAN]
CF = np.array(df_CF[PLAN])
Time_CF = df_CF.index
DF = np.array(df_CF['Time'])
CF

In [ ]:
#from PyCurve.curve import Curve

In [ ]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 145)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


In [ ]:
raw_liab_ibt_dict={}
tr = Raw_FTSE_YC['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict[col])
#curve = Curve(t,y)
#print("curve.get_rate:{},curve.get_time:{}".format(curve.get_rate,curve.get_time))
#print(curve.get_rate)
#print(curve.get_time)

In [ ]:
int_df = pd.DataFrame(raw_liab_ibt_dict)
int_df = int_df.drop(['Date'], axis=1)
int_df = int_df.iloc[:, ::-1]
int_df

In [ ]:
liab_plan_dict={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df[col][j]/100)**DF[j]))
    liab_plan_dict[col] = temp_pv
len(liab_plan_dict)

In [ ]:
pv_df = pd.DataFrame(liab_plan_dict, index = ['Present Values']).transpose()
np.negative(pv_df['Present Values'])
pv_df

In [ ]:
Plan_Return_YC = np.zeros(len(liab_plan_dict)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC[i] = ((pv_df['Present Values'][i+1])/pv_df['Present Values'][i])-1
Plan_Return_YC

In [ ]:
Plan_Returns_YC = pd.DataFrame(Plan_Return_YC)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC.to_excel(writer, sheet_name='returns')
pv_df.to_excel(writer, sheet_name='pv')
writer.save()

In [ ]:
df_SC = pd.read_excel("2021_PBO_CF.xlsx",sheet_name = "Service Cost Cashflows Summary", skiprows = [0,1,2], usecols=[0,1,2,3], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC

In [ ]:
df_SC = df_SC.divide(12)
df_SC

In [ ]:
df=df_SC.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_SC = df
df_SC

In [ ]:
df_SC_add = pd.read_excel("2021 Liability Cash Flows.xlsx",sheet_name = "SC", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_SC_add

In [ ]:
df_SC  = df_SC_add.append(df_SC)
df_SC

In [ ]:
col_SC = df_SC.columns
col_CF = df_CF.columns
df_CF_Tot = UPS_Contr_Pctg*df_SC[col_SC] + df_CF[col_CF]
df_CF_Tot

In [ ]:
df_CF_Tot['Time'] = df_t.values
df_CF_Tot

In [ ]:
CF_Tot = df_CF_Tot[PLAN]
Time_CF_Tot = df_CF_Tot.index
DF_Tot=df_CF_Tot['Time']

In [ ]:
liab_plan_dict_tot={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF_Tot[j]/((1+int_df[col][j]/100)**DF_Tot[j]))
    liab_plan_dict_tot[col] = temp_pv
liab_plan_dict_tot


In [ ]:
pv_df_tot = pd.DataFrame(liab_plan_dict_tot, index = ['Present Values']).transpose()

pv_df_tot

In [ ]:
Plan_Return_YC_Tot = np.zeros(len(liab_plan_dict_tot)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_tot)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Tot[i] = ((pv_df_tot['Present Values'][i+1])/pv_df_tot['Present Values'][i])-1
Plan_Returns_YC_Tot = pd.DataFrame(Plan_Return_YC_Tot)
Plan_Returns_YC_Tot

In [ ]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability & SC Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Tot.to_excel(writer, sheet_name='returns')
pv_df_tot.to_excel(writer, sheet_name='pv')
writer.save()

Calculate IRR's

In [ ]:
Plan_DR = pd.read_excel("UPS Pension - Historical Liability Data - 9.30.21.xlsx",sheet_name = PLAN ,skiprows = [0,1,2,3],usecols=[1,2], na_values=[""],index_col=0)
#dataset.set_index('Year')
Plan_DR


In [ ]:
#col_DR = Plan_DR.columns
#df_DR_M =  (1+Plan_DR)**(1/12)-1
#df_DR_M = Plan_DR
IRR = Plan_DR['IRR']
IRR

In [ ]:
#liab_plan_dict_dr={}
#for col in Plan_DR.columns:
    #print('col: {}'.format(col))
#temp_pv=0
for j in range (len(Plan_DR)):
    for i in range (len(Time_CF)):
    #for j in range (0,len(Time_CF)):
        temp_pv += (CF[i]/((1+Plan_DR['IRR'][j]**DF[i])))
    #liab_plan_dict_dr[col] = temp_pv
    PV_DR = temp_pv
PV_DR

In [ ]:
temp_pv = (CF[0]/((1+Plan_DR['IRR'][-1])**DF[0]))
temp_pv

In [ ]:
Plan_PV_DR = np.zeros(len(Plan_DR))
for j in range (len(Plan_DR)):
    for i in range (len(Time_CF)):
        Plan_PV_DR[j] += (CF[i]/((1+Plan_DR['IRR'][j])**DF[i]))
Plan_PV_DR

In [ ]:
pv_df_dr = pd.DataFrame(Plan_PV_DR, columns=['Present Values'])
#np.negative(pv_df_dr['Present Values'])
test = np.append(np.negative(pv_df_dr['Present Values'][0]),CF)
#pv_df_dr['Present Values']
test

In [ ]:
#pv_df_dr = pd.DataFrame(Plan_PV_DR)
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability PV Using DR.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
pv_df_dr.to_excel(writer, sheet_name='Present Values')
writer.save()

In [ ]:
import numpy as np
from scipy.optimize import fsolve

def npv(irr, cfs, yrs):  
    return np.sum(cfs / (1. + irr) ** yrs)

def irr(cfs, yrs, x0, **kwargs):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs), **kwargs))

In [ ]:
IRR = np.zeros(len(pv_df))
for j in range (len(pv_df)):
    cash_flow = np.append(np.negative(pv_df['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

In [ ]:
IRR = np.zeros(len(pv_df_tot))
for j in range (len(pv_df_tot)):
    cash_flow = np.append(np.negative(pv_df_tot['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

In [ ]:
IRR = np.zeros(len(Plan_DR))
for j in range (len(Plan_DR)):
    cash_flow = np.append(np.negative(pv_df_dr['Present Values'][j]),CF)
    years_ago = np.append(0,DF)
    IRR[j] = irr(cash_flow, years_ago, x0=0.10, maxfev=10000)
IRR

In [ ]:
df_Raw_FTSE_YC_Pre2010 = pd.read_excel("Raw FTSE Data pre 2010.xlsx",skiprows = [0,2],usecols = [*range(14, 185)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC_Pre2010 = df_Raw_FTSE_YC_Pre2010.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC_Pre2010.columns
Raw_FTSE_YC_Pre2010


In [ ]:
Raw_FTSE_YC_Tot = Raw_FTSE_YC.join(Raw_FTSE_YC_Pre2010)
Raw_FTSE_YC_Tot

In [ ]:
raw_liab_ibt_dict_tot={}
tr = Raw_FTSE_YC_Tot['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC_Tot[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC_Tot.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC_Tot[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict_tot[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict_tot[col])

In [ ]:
int_df_tot = pd.DataFrame(raw_liab_ibt_dict_tot)
int_df_tot = int_df_tot.drop(['Date'], axis=1)
int_df_tot = int_df_tot.iloc[:, ::-1]
int_df_tot

In [ ]:
liab_plan_dict_totdf={}
for col in int_df_tot.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df_tot[col][j]/100)**DF[j]))
    liab_plan_dict_totdf[col] = temp_pv
liab_plan_dict_totdf

In [ ]:
pv_df_totdf= pd.DataFrame(liab_plan_dict_totdf, index = ['Present Values']).transpose()
np.negative(pv_df_totdf['Present Values'])
pv_df_totdf.tail

In [ ]:
Plan_Return_YC_Totdf = np.zeros(len(liab_plan_dict_totdf)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_totdf)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf[i] = ((pv_df_totdf['Present Values'][i+1])/pv_df_totdf['Present Values'][i])-1
Plan_Returns_YC_Totdf = pd.DataFrame(Plan_Return_YC_Totdf)
Plan_Returns_YC_Totdf

In [ ]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV Pre 2010.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Totdf.to_excel(writer, sheet_name='returns')
pv_df_totdf.to_excel(writer, sheet_name='pv')
writer.save()

Test Liability Return Data Pre-2010

In [ ]:
df_Raw_FTSE_YC = pd.read_excel("Raw FTSE Data.xlsx",skiprows = [0,2],usecols = [*range(0, 130)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC = df_Raw_FTSE_YC.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC.columns
Raw_FTSE_YC


In [ ]:
df_Raw_FTSE_YC_Pre2010 = pd.read_excel("Raw FTSE Data pre 2010.xlsx",skiprows = [0,2],usecols = [*range(1, 185)], na_values=[""])
#dataset.set_index('Date')
Raw_FTSE_YC_Pre2010 = df_Raw_FTSE_YC_Pre2010.copy()
#Raw_FTSE_YC = pd.DataFrame(Raw_FTSE_YC)
#df_Raw_FTSE_YC.set_index('Date')
col = Raw_FTSE_YC_Pre2010.columns
Raw_FTSE_YC_Pre2010


In [ ]:
Raw_FTSE_YC_Tot_1 = Raw_FTSE_YC.join(Raw_FTSE_YC_Pre2010)
Raw_FTSE_YC_Tot_1

In [ ]:
raw_liab_ibt_dict_tot_1={}
tr = Raw_FTSE_YC_Tot_1['Date']
# list of raw (not interpolated) times to maturity
yr = Raw_FTSE_YC_Tot_1[col] 
# list of raw (not interpolated) yields
t = list(frange(0.5, 30.08, (1/12))) # interpolating in range 1..30 years

for col in Raw_FTSE_YC_Tot_1.columns:
    #print('col: {}'.format(col))
    y = []
    interp = sp.interpolate.interp1d(tr, Raw_FTSE_YC_Tot_1[col] , bounds_error=False, fill_value=sp.nan)
#for col in Raw_FTSE_YC.columns:
    #for col in FTSE_YC.columns:
    for i in t:
            value = float(interp(i))
            if not sp.isnan(value): # Don't include out-of-range values
                y.append(value)
                End_Rate = [y[-1]] * 592
                Beg_Rate = [y[0]] * 5
            raw_liab_ibt_dict_tot_1[col] = Beg_Rate + y + End_Rate
len(raw_liab_ibt_dict_tot_1[col])

In [ ]:
int_df_tot_1 = pd.DataFrame(raw_liab_ibt_dict_tot_1)
int_df_tot_1 = int_df_tot_1.drop(['Date'], axis=1)
int_df_tot_1 = int_df_tot_1.iloc[:, ::-1]
int_df_tot_1[col]

In [ ]:
liab_plan_dict_totdf_1={}
for col in int_df_tot_1.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF[j]/((1+int_df_tot_1[col][j]/100)**DF[j]))
    liab_plan_dict_totdf_1[col] = temp_pv
liab_plan_dict_totdf_1

In [ ]:
pv_df_totdf_1= pd.DataFrame(liab_plan_dict_totdf_1, index = ['Present Values']).transpose()
np.negative(pv_df_totdf_1['Present Values'])
pv_df_totdf_1

In [ ]:
Plan_Return_YC_Totdf_1 = np.zeros(len(liab_plan_dict_totdf_1)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_totdf_1)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf_1[i] = ((pv_df_totdf_1['Present Values'][i+1])/pv_df_totdf_1['Present Values'][i])-1
Plan_Returns_YC_Totdf_1 = pd.DataFrame(Plan_Return_YC_Totdf_1)
Plan_Returns_YC_Totdf_1

In [ ]:
#export monthly dataframe to excel
filepath = (PLAN+ ' Liability Returns & PV Pre 2010_1.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Returns_YC_Totdf_1.to_excel(writer, sheet_name='returns')
pv_df_totdf_1.to_excel(writer, sheet_name='pv')
writer.save()

In [ ]:
Plan_PV_DR_ERF = np.zeros(len(df_t))
for j in range (len(df_t)):
    for i in range (j,len(Time_CF_Tot)):
        Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
Plan_PV_DR_ERF[240]

In [ ]:
#fullfillment_return = .045
#MV_Assets = 13203547000


In [ ]:
#Plan_Asset_MV = np.zeros(len(df_t))
#for i in range(len(df_t)):
       #if (i == 0):
            
                #Plan_Asset_MV[i] = MV_Assets
        #else:
            
                #Plan_Asset_MV[i] = (Plan_Asset_MV[i-1]*(1+fullfillment_return)**DF[0].tolist())-CF[i-1]
#Plan_Asset_MV

In [ ]:
#df_Asset_MV = pd.DataFrame(Plan_Asset_MV)
#df_Asset_MV

In [ ]:
#FR = Plan_Asset_MV/Plan_PV_DR_ERF
#FR = pd.DataFrame(FR)
#FR.style.format("{:.2%}")


In [ ]:
#FS = Plan_Asset_MV[-1] - Plan_PV_DR_ERF[-1] 
#FS

In [ ]:
def full_solve(x0):
    return np.asscalar(fsolve(fullfillment_solve, x0=x0))


def fullfillment_solve(fullfillment_return):
    #fullfillment_return = .045
    MV_Assets = 13203547000
    Plan_PV_DR_ERF = np.zeros(len(df_t))
    Plan_Asset_MV = np.zeros(len(df_t))
    x = Years_to_FF/DF[0]
    x = x.astype(int)
    
    for j in range(len(df_t)):
        if (j == 0):
            for i in range(j,len(Time_CF_Tot)):
                Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
                Plan_Asset_MV[j] = MV_Assets
    
        else:
            for i in range(j,len(Time_CF_Tot)):
                Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
                Plan_Asset_MV[j] = (Plan_Asset_MV[j-1]*(1+fullfillment_return)**DF[0].tolist())-CF[j-1]
    
    return Plan_Asset_MV[x] - Plan_PV_DR_ERF[x]*FF_FR


    

In [ ]:
fullfill_irr = full_solve(x0=.01)
print("Base Case Fullfillment rate of return:%3.8f"%fullfill_irr)


In [ ]:
fullfillment_return = fullfill_irr
MV_Assets = 13203547000
Plan_PV_DR_ERF = np.zeros(len(df_t))
Plan_Asset_MV = np.zeros(len(df_t))
x = Years_to_FF/DF[0]
x = x.astype(int)
for j in range(len(df_t)):
    if (j == 0):
        for i in range(j,len(Time_CF_Tot)):
            Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
            Plan_Asset_MV[j] = MV_Assets
    
    else:
        for i in range(j,len(Time_CF_Tot)):
            Plan_PV_DR_ERF[j] += (CF_Tot[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
            Plan_Asset_MV[j] = (Plan_Asset_MV[j-1]*(1+fullfillment_return)**DF[0].tolist())-CF[j-1]
    
Plan_Asset_MV[x] - Plan_PV_DR_ERF[x]*FF_FR

In [ ]:
Excess_Return = fullfill_irr - Plan_DR['IRR'][-1]
#Excess_Return = pd.DataFrame(Excess_Return)
#Excess_Return.style.format("{:.2%}")
Excess_Return

In [ ]:
df_PVFB = pd.read_excel("YE2020 PBO SC Cashflows_QP.xlsx",sheet_name = "PVFB Cashflows Summary", skiprows = [0,1,2,3,4,5,6,7,8], usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_PVFB

In [ ]:
df_PVFB = df_PVFB.divide(12)
df_PVFB

In [ ]:
df=df_PVFB.copy()

#set start date and end date
start_date = df.index.min() - pd.DateOffset(day=0)
end_date = df.index.max() + pd.DateOffset(day=31)

#create new dataframe monthly index
dates = pd.date_range(start_date, end_date, freq='M')
dates.name = 'date'

#reindex yearly dataframe to monthly dataframe
df = df.reindex(dates, method='ffill')

#export monthly dataframe to excel
#filepath = 'monthly_data.xlsx'
#writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='data')
#writer.save()
df_PVFB = df
df_PVFB

In [ ]:
df_PVFB_add = pd.read_excel("YE2020 PBO SC Cashflows_QP.xlsx",sheet_name = "PVFB Cashflows Summary 2021", usecols=[0,1,2,3,4], na_values=[""],index_col=0)
#dataset.set_index('Year')
df_PVFB_add

In [ ]:
df_PVFB  = df_PVFB_add.append(df_PVFB)
df_PVFB

In [ ]:
df_PVFB = df_PVFB.subtract(df_CF)
df_PVFB

In [ ]:
col_PVFB = df_PVFB.columns
col_CF = df_CF.columns
df_CF_Tot_PVFB = UPS_Contr_Pctg*df_PVFB[col_PVFB] + df_CF[col_CF]
df_CF_Tot_PVFB

In [ ]:
df_CF_Tot_PVFB['Time'] = df_t.values
df_CF_Tot_PVFB

In [ ]:
CF_Tot_PVFB = df_CF_Tot_PVFB[PLAN]
Time_CF_Tot_PVFB = df_CF_Tot_PVFB.index
DF_Tot_PVFB=df_CF_Tot_PVFB['Time']

In [ ]:
liab_plan_dict_tot_pvfb={}
for col in int_df.columns:
    #print('col: {}'.format(col))
    temp_pv=0
    for j in range (0,len(Time_CF)):
        temp_pv += (CF_Tot_PVFB[j]/((1+int_df[col][j]/100)**DF_Tot_PVFB[j]))
    liab_plan_dict_tot_pvfb[col] = temp_pv
liab_plan_dict_tot_pvfb


In [ ]:
pv_df_tot_pvfb = pd.DataFrame(liab_plan_dict_tot_pvfb, index = ['Present Values']).transpose()

pv_df_tot_pvfb

In [ ]:
Plan_Return_YC_Totdf_pvfb = np.zeros(len(liab_plan_dict_tot_pvfb)-1)
#for j in range (0,len(Time_DR)):
for i in range (0,len(liab_plan_dict_tot_pvfb)-1):
    #print(i)
#     print(IBT_Return_YC[i])
    Plan_Return_YC_Totdf_pvfb[i] = ((pv_df_tot_pvfb['Present Values'][i+1])/pv_df_tot_pvfb['Present Values'][i])-1
Plan_Return_YC_Totdf_pvfb = pd.DataFrame(Plan_Return_YC_Totdf_pvfb)
Plan_Return_YC_Totdf_pvfb

In [ ]:
#export monthly dataframe to excel
filepath = (PLAN+ ' PVFB Liability Returns & PV.xlsx')
writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
Plan_Return_YC_Totdf_pvfb.to_excel(writer, sheet_name='returns')
pv_df_tot_pvfb.to_excel(writer, sheet_name='pv')
writer.save()

In [ ]:
def full_solve(x0):
    return np.asscalar(fsolve(fullfillment_solve, x0=x0))


def fullfillment_solve(fullfillment_return):
    #fullfillment_return = .045
    MV_Assets = 13203547000
    Plan_PV_DR_ERF = np.zeros(len(df_t))
    Plan_Asset_MV = np.zeros(len(df_t))
    x = Years_to_FF/DF[0]
    x = x.astype(int)
    
    for j in range(len(df_t)):
        if (j == 0):
            for i in range(j,len(Time_CF_Tot_PVFB)):
                Plan_PV_DR_ERF[j] += (CF_Tot_PVFB[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
                Plan_Asset_MV[j] = MV_Assets
    
        else:
            for i in range(j,len(Time_CF_Tot_PVFB)):
                Plan_PV_DR_ERF[j] += (CF_Tot_PVFB[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
                Plan_Asset_MV[j] = (Plan_Asset_MV[j-1]*(1+fullfillment_return)**DF[0].tolist())-CF_Tot_PVFB[j-1]
    
    return Plan_Asset_MV[x] - Plan_PV_DR_ERF[x]*FF_FR


In [ ]:
fullfill_irr = full_solve(x0=.01)
print("Base Case Fullfillment rate of return:%3.8f"%fullfill_irr)


In [ ]:
fullfillment_return = fullfill_irr
MV_Assets = 13203547000
Plan_PV_DR_ERF = np.zeros(len(df_t))
Plan_Asset_MV = np.zeros(len(df_t))
x = Years_to_FF/DF[0]
x = x.astype(int)
for j in range(len(df_t)):
    if (j == 0):
        for i in range(j,len(Time_CF_Tot_PVFB)):
            Plan_PV_DR_ERF[j] += (CF_Tot_PVFB[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
            Plan_Asset_MV[j] = MV_Assets
    
    else:
        for i in range(j,len(Time_CF_Tot_PVFB)):
            Plan_PV_DR_ERF[j] += (CF_Tot_PVFB[i]/((1+Plan_DR['IRR'][-1])**DF[i-j]))
            Plan_Asset_MV[j] = (Plan_Asset_MV[j-1]*(1+fullfillment_return)**DF[0].tolist())-CF_Tot_PVFB[j-1]
    
Plan_Asset_MV[x] - Plan_PV_DR_ERF[x]*FF_FR

In [ ]:
Excess_Return = fullfill_irr - Plan_DR['IRR'][-1]
#Excess_Return = pd.DataFrame(Excess_Return)
#Excess_Return.style.format("{:.2%}")
Excess_Return